In [1]:
from taxcalc import *
import pandas as pd
import behresp as br

In [2]:
ref_0={}
ref_1={
    "II_brk5": {2021: [211844, 400000, 211844, 211844, 400000],
                2026: [502101, 502101, 251050, 502101, 502101]},
    "II_rt6": {2021: 0.33,
               2026: 0.35},
    "II_brk6": {2021: [416700, 416700, 317775, 416700, 416700],
               2026: [504149, 567168, 283575, 535659, 567168]},
    "II_rt7": {2021: 0.35,
               2026: 0.396},
    "II_brk7": {2021: [418400, 470700, 400000, 444550, 470700],
                2026: [9e99, 9e99, 9e99, 9e99, 9e99]},
    "II_rt8": {2021: 0.396,
               2026: 1},
    "PT_brk5": {2021: [211844, 400000, 211844, 211844, 400000],
                2026: [502101, 502101, 251050, 502101, 502101]},
    "PT_rt6": {2021: 0.33,
               2026: 0.35},
    "PT_brk6": {2021: [416700, 416700, 317775, 416700, 416700],
               2026: [504149, 567168, 283575, 535659, 567168]},
    "PT_rt7": {2021: 0.35,
               2026: 0.396},
    "PT_brk7": {2021: [418400, 470700, 400000, 444550, 470700],
                2026: [9e99, 9e99, 9e99, 9e99, 9e99]},
    "PT_rt8": {2021: 0.396,
               2026: 1}
}
ref_2={
    "ID_prt": {2021: 0.03},
    "ID_ps": {2021: [400000, 400000, 400000, 400000, 400000]}
}
ref_3={
    "CG_rt4": {2021: 0.396},
    "CG_brk3": {2021: [1000000, 1000000, 1000000, 1000000, 1000000]}
}
ref_4={"EITC_MaxEligAge": {2021: 125}}
ref_5={
    "ID_BenefitCap_rt": {2021: 0.28,
                         2026: 1},
    "ID_BenefitCap_Switch": {2021: [True, True, False, True, True, True, True],
                             2026: [True, True, True, True, True, True, True]}
}
ref_6={"SS_Earnings_thd": {2021: 400000}}

In [3]:
refs_biden = [ref_1, ref_2, ref_3, ref_4, ref_5, ref_6]
refs_base = [ref_0, ref_1, ref_2, ref_3, ref_4, ref_5]
rec = Records()

df = pd.DataFrame(index=["Restore pre-TCJA rates above $400,000 of income",
                         "Restore limitation on itemized deductions above $400,000 of income",
                         "Tax capital gains and dividends at same rate as ordinary income above $1 million of income",
                         "Extend the EITC to childless workers age 65 and older",
                         "Limit tax benefit of itemized deductions to 28 percent of value",
                         "Apply 12.4 percent OASDI payroll tax to eranings above $400,000"])
for year in range(2021, 2031):
    pol_base = Policy()
    pol_ref = Policy()
    col=[]
    # NOTE: the assumed capital gains response elasticity has a huge effect on revenue.
    # In its report, TPC assumes a lower response elasticity because "the Biden plan taxes
    # unrealized capital gains at death, which reduces the incentive to defer realizations of gains."
    # Since Tax-Calculator is not able to estimate revenue from taxing unrealized capital gains
    # at death, the response elasticity here is higher than TPC's and results in
    # an overall reduction in revenue collected.
    response_elasticities = {'sub': 0, 'inc': 0, 'cg': -3}
    # "stack" reform provisions in the same order as TPC
    for (ref_biden, ref_base) in zip(refs_biden, refs_base): 
        pol_base.implement_reform(ref_base)
        calc_base=Calculator(pol_base, rec)
        calc_base.advance_to_year(year)
        
        pol_ref.implement_reform(ref_biden)
        calc_ref=Calculator(pol_ref, rec)
        calc_ref.advance_to_year(year)
        
        base_df, ref_df = br.response(calc_base, calc_ref, response_elasticities)
        base_rev= (base_df['combined'] * base_df['s006']).sum()/1e9
        ref_rev = (ref_df['combined'] * ref_df['s006']).sum()/1e9
        
        dif_rev = ref_rev - base_rev
        col.append(round(dif_rev,2))
    df[year] = col
        

In [4]:
df_final = df.append(df.sum().rename('Total'))
df_final

,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030
"Restore pre-TCJA rates above $400,000 of income",33.60,35.02,36.56,38.18,39.76,-0.00,-0.00,-0.00,-0.00,-0.00
"Restore limitation on itemized deductions above $400,000 of income",11.36,11.77,12.26,12.76,13.29,-1.29,-1.36,-1.41,-1.49,-1.51
Tax capital gains and dividends at same rate as ordinary income above $1 million of income,-14.46,-13.67,-13.22,-12.58,-12.70,-17.40,-17.93,-18.30,-18.77,-19.13
Extend the EITC to childless workers age 65 and older,-0.51,-0.54,-0.56,-0.58,-0.61,-0.63,-0.66,-0.67,-0.69,-0.70
Limit tax benefit of itemized deductions to 28 percent of value,1.39,1.47,1.52,1.58,1.63,0.00,0.00,0.00,0.00,0.00
"Apply 12.4 percent OASDI payroll tax to eranings above $400,000",83.10,88.65,94.25,100.02,106.08,112.55,119.84,127.71,135.82,144.05
Total,114.48,122.70,130.81,139.38,147.45,93.23,99.89,107.33,114.87,122.71
